In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,18412
2,Huelva,Confirmados PDIA 14 días,5437
3,Huelva,Tasa PDIA 14 días,"1064,5275608280485"
4,Huelva,Confirmados PDIA 7 días,2824
5,Huelva,Total Confirmados,18625
6,Huelva,Curados,10838
7,Huelva,Fallecidos,159


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  18412.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  4945.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 232 personas en los últimos 7 días 

Un positivo PCR cada 115 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,18412.0,2824.0,5437.0,510743.0,552.919962,1064.527561,1316.0
Huelva-Costa,11177.0,1967.0,3686.0,288115.0,682.713500,1279.350259,851.0
Condado-Campiña,4986.0,640.0,1176.0,155057.0,412.751440,758.430771,362.0
Huelva (capital),4945.0,618.0,1252.0,143663.0,430.173392,871.483959,291.0
Isla Cristina,1559.0,390.0,851.0,21264.0,1834.085779,4002.069225,131.0
Lepe,1488.0,261.0,414.0,27431.0,951.478255,1509.241369,114.0
Ayamonte,790.0,249.0,401.0,20946.0,1188.771126,1914.446672,104.0
Sierra de Huelva-Andévalo Central,1999.0,212.0,507.0,67571.0,313.744062,750.321884,102.0
Punta Umbría,439.0,134.0,175.0,15242.0,879.149718,1148.143288,72.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villalba del Alcor,354.0,76.0,210.0,3338.0,2276.812463,6291.192331,38.0
Encinasola,109.0,14.0,71.0,1305.0,1072.796935,5440.613027,5.0
Villanueva de las Cruces,16.0,8.0,16.0,373.0,2144.772118,4289.544236,1.0
Isla Cristina,1559.0,390.0,851.0,21264.0,1834.085779,4002.069225,131.0
Berrocal,10.0,5.0,10.0,312.0,1602.564103,3205.128205,3.0
Calañas,142.0,25.0,84.0,2791.0,895.736295,3009.673952,4.0
Paymogo,38.0,19.0,26.0,1159.0,1639.344262,2243.313201,6.0
Cerro de Andévalo (El),115.0,18.0,50.0,2364.0,761.421320,2115.059222,10.0
Ayamonte,790.0,249.0,401.0,20946.0,1188.771126,1914.446672,104.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aroche,79.0,1.0,7.0,3073.0,32.541490,227.790433,0.0,0.142857
Minas de Riotinto,94.0,4.0,21.0,3848.0,103.950104,545.738046,1.0,0.190476
Encinasola,109.0,14.0,71.0,1305.0,1072.796935,5440.613027,5.0,0.197183
Villarrasa,116.0,5.0,24.0,2176.0,229.779412,1102.941176,4.0,0.208333
Castaño del Robledo,7.0,1.0,4.0,223.0,448.430493,1793.721973,0.0,0.250000
Zarza-Perrunal (La),19.0,1.0,4.0,1267.0,78.926598,315.706393,1.0,0.250000
Calañas,142.0,25.0,84.0,2791.0,895.736295,3009.673952,4.0,0.297619
Cerro de Andévalo (El),115.0,18.0,50.0,2364.0,761.421320,2115.059222,10.0,0.360000
Villalba del Alcor,354.0,76.0,210.0,3338.0,2276.812463,6291.192331,38.0,0.361905
